In [ ]:
import cv2
import win32gui
import win32con

# 프레임 생성 및 초기 설정
frame_name = "CCTV Viewer"
frame_width, frame_height = 800, 600
frame = cv2.namedWindow(frame_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(frame_name, frame_width, frame_height)
cv2.setWindowProperty(frame_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# 16분할로 나눌 화면 크기 설정
screen_width, screen_height = frame_width // 4, frame_height // 4

# CCTV 채널 정보
channel_info = [
    "Channel 1", "Channel 2", "Channel 3", "Channel 4",
    "Channel 5", "Channel 6", "Channel 7", "Channel 8",
    "Channel 9", "Channel 10", "Channel 11", "Channel 12",
    "Channel 13", "Channel 14", "Channel 15", "Channel 16"
]

# CCTV 화면 로드
cctv_screen = cv2.imread("cctv_screen.png")

while True:
    # 프레임 초기화
    frame_img = cv2.imread("background.png")  # 프레임에 사용할 배경 이미지로 background.png를 사용합니다.

    # CCTV 화면 16분할로 표시
    for i in range(4):
        for j in range(4):
            x = j * screen_width
            y = i * screen_height
            screen = cctv_screen[y:y+screen_height, x:x+screen_width]
            frame_img[y:y+screen_height, x:x+screen_width] = screen

            # 채널 정보 표시
            channel_x = x + 10
            channel_y = y + screen_height - 30
            cv2.putText(frame_img, channel_info[i*4 + j], (channel_x, channel_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # 프레임에 투명도 적용
    frame_img = cv2.cvtColor(frame_img, cv2.COLOR_BGR2BGRA)
    frame_img[:, :, 3] = 100  # 투명도 값을 조절하여 원하는 수준으로 설정합니다.

    # 프레임 표시
    cv2.imshow(frame_name, frame_img)

    # 종료 조건 설정 (ESC 키를 누르면 종료)
    if cv2.waitKey(1) == 27:
        break

# 리소스 해제
cv2.destroyAllWindows()


In [ ]:
import cv2
import win32gui
import win32con
import pygetwindow as gw

# 프레임 생성 및 초기 설정
frame_name = "Transparent Frame"
frame_width, frame_height = 800, 600
frame = cv2.namedWindow(frame_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(frame_name, frame_width, frame_height)

# 16분할로 나눌 화면 크기 설정
screen_width, screen_height = frame_width // 4, frame_height // 4

# 윈도우 바탕화면 찾기
desktop = gw.getWindowsWithTitle("Program Manager")[0]
desktop_hwnd = desktop._hWnd

# CCTV 채널 정보
channel_info = [
    "Channel 1", "Channel 2", "Channel 3", "Channel 4",
    "Channel 5", "Channel 6", "Channel 7", "Channel 8",
    "Channel 9", "Channel 10", "Channel 11", "Channel 12",
    "Channel 13", "Channel 14", "Channel 15", "Channel 16"
]

while True:
    # 프레임 초기화
    frame_img = cv2.imread("background.png")  # 프레임에 사용할 배경 이미지로 background.png를 사용합니다.

    # 윈도우 바탕화면 이미지 가져오기
    hwnd = win32gui.GetDesktopWindow()
    window_rect = win32gui.GetWindowRect(hwnd)
    window_width = window_rect[2] - window_rect[0]
    window_height = window_rect[3] - window_rect[1]
    hDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(hDC)
    memDC = dcObj.CreateCompatibleDC()
    screenshot = win32ui.CreateBitmap()
    screenshot.CreateCompatibleBitmap(dcObj, window_width, window_height)
    memDC.SelectObject(screenshot)
    memDC.BitBlt((0, 0), (window_width, window_height), dcObj, (0, 0), win32con.SRCCOPY)
    screenshot_img = np.frombuffer(screenshot.GetBitmapBits(True), dtype=np.uint8)
    screenshot_img.shape = (window_height, window_width, 4)

    # 윈도우 바탕화면 투명도 적용
    screenshot_img = cv2.cvtColor(screenshot_img, cv2.COLOR_BGRA2RGBA)
    screenshot_img[:, :, 3] = 100  # 투명도 값을 조절하여 원하는 수준으로 설정합니다.

    # 윈도우 바탕화면을 16분할로 표시
    for i in range(4):
        for j in range(4):
            x = j * screen_width
            y = i * screen_height
            screen = screenshot_img[y:y+screen_height, x:x+screen_width]
            frame_img[y:y+screen_height, x:x+screen_width] = screen

            # 채널 정보 표시
            channel_x = x + 10
            channel_y = y + screen_height - 30
            cv2.putText(frame_img, channel_info[i*4 + j], (channel_x, channel_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # 프레임 표시
    cv2.imshow(frame_name, frame_img)

    # 윈도우 바탕화면 업데이트
    bmpinfo = screenshot.GetInfo()
    screenshot_img = screenshot_img[:, :, :3]  # 알파 채널 제거
    screenshot.SetBitmapBits(screenshot_img.tobytes())

    # 윈도우 크기 조정 (윈도우 바탕화면 크기와 일치)
    win32gui.SetWindowPos(frame_name, win32con.HWND_TOPMOST, window_rect[0], window_rect[1], window_width, window_height, win32con.SWP_SHOWWINDOW)

    # 종료 조건 설정 (ESC 키를 누르면 종료)
    if cv2.waitKey(1) == 27:
        break

# 리소스 해제
cv2.destroyAllWindows()


In [ ]:
import pandas as pd

# 액셀 파일들의 경로
file_paths = ['tttm1.xlsx', 'tttm2.xlsx']
# file_paths = ['file1.xlsx', 'file2.xlsx', 'file3.xlsx', 'file4.xlsx', 'file5.xlsx', 'file6.xlsx']

# 데이터프레임들을 담을 리스트
dfs = []

# 액셀 파일들을 읽어서 데이터프레임으로 변환
for file_path in file_paths:
    df = pd.read_excel(file_path)
    dfs.append(df)

# 첫 번째 데이터프레임을 기준으로 비교
base_df = dfs[0]

# 매칭되지 않는 값들을 담을 데이터프레임
mismatch_df = pd.DataFrame()

# 기준 데이터프레임과 나머지 데이터프레임들을 비교
for df in dfs[1:]:
    # 매칭되지 않는 값을 찾음
    mismatch = base_df.merge(df, indicator=True, how='left').loc[lambda x: x['_merge'] == 'left_only']
    mismatch_df = mismatch_df.append(mismatch)


# CSV 파일로 저장
mismatch_df.to_excel('mismatch.xlsx', index=False)


In [ ]:
# 이거다 굿굿

import pandas as pd

# 액셀 파일들의 경로
file_paths = ['tttm1.xlsx', 'tttm2.xlsx', 'tttm3.xlsx']

# 액셀 파일을 데이터프레임으로 읽어오는 함수
def read_excel(file_path):
    return pd.read_excel(file_path)

# 비교할 액셀 파일들을 데이터프레임으로 읽어오기
dfs = [read_excel(file_path) for file_path in file_paths]

# 비교 대상 액셀 파일의 id 컬럼을 기준으로 데이터프레임을 병합
compare_df = pd.concat([df.set_index('id') for df in dfs], axis=1, keys=file_paths)

# 비교 결과를 "compare.xlsx" 파일로 저장
compare_df.to_excel('compare.xlsx')




In [ ]:
# 최종값이 다르면 표시해주는거 , 빈쎌은 n/a로 표시

import pandas as pd

# 액셀 파일들의 경로
file_paths = ['tttm1.xlsx', 'tttm2.xlsx', 'tttm3.xlsx']

# 액셀 파일을 데이터프레임으로 읽어오는 함수
def read_excel(file_path):
    return pd.read_excel(file_path)

# 비교할 액셀 파일들을 데이터프레임으로 읽어오기
dfs = [read_excel(file_path) for file_path in file_paths]

# 비교 대상 액셀 파일의 id 컬럼을 기준으로 데이터프레임을 병합
compare_df = pd.concat([df.set_index('id') for df in dfs], axis=1, keys=file_paths)

# NaN 값을 'N/A'로 변환
compare_df = compare_df.fillna('N/A')

# 각 행을 비교하여 결과 열 추가
compare_df['결과'] = compare_df.apply(lambda row: '일치' if row.nunique() == 1 else '값이다름', axis=1)

# 비교 결과를 "compare.xlsx" 파일로 저장
compare_df.to_excel('compare.xlsx')



In [ ]:
# 파일을 선택할 수 있게

import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilenames

# Tkinter 루트 창 초기화
root = Tk()
root.withdraw()

# 파일 선택 대화상자 표시
file_paths = askopenfilenames(title='액셀 파일 선택', filetypes=[('Excel files', '*.xlsx')])

# 액셀 파일을 데이터프레임으로 읽어오는 함수
def read_excel(file_path):
    return pd.read_excel(file_path)

# 비교할 액셀 파일들을 데이터프레임으로 읽어오기
dfs = [read_excel(file_path) for file_path in file_paths]

# 비교 대상 액셀 파일의 id 컬럼을 기준으로 데이터프레임을 병합
compare_df = pd.concat([df.set_index('id') for df in dfs], axis=1, keys=file_paths)

# NaN 값을 'N/A'로 변환
compare_df = compare_df.fillna('N/A')

# 각 행을 비교하여 결과 열 추가
compare_df['결과'] = compare_df.apply(lambda row: '일치' if row.nunique() == 1 else '값이다름', axis=1)

# 비교 결과를 "compare.xlsx" 파일로 저장
compare_df.to_excel('compare.xlsx')


In [5]:
# 파일 선택창을 맨위에 뜨게

import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilenames
from tkinter.messagebox import showinfo

# Tkinter 루트 창 초기화
root = Tk()
root.withdraw()
root.attributes('-topmost', True)  # 창을 최상위로 설정

# 파일 선택 대화상자 표시
file_paths = askopenfilenames(title='액셀 파일 선택', filetypes=[('Excel files', '*.xlsx')])

# 액셀 파일을 데이터프레임으로 읽어오는 함수
def read_excel(file_path):
    return pd.read_excel(file_path)

# 비교할 액셀 파일들을 데이터프레임으로 읽어오기
dfs = [read_excel(file_path) for file_path in file_paths]

# 비교 대상 액셀 파일의 id 컬럼을 기준으로 데이터프레임을 병합
compare_df = pd.concat([df.set_index('id') for df in dfs], axis=1, keys=file_paths)

# NaN 값을 'N/A'로 변환
compare_df = compare_df.fillna('N/A')

# 각 행을 비교하여 결과 열 추가
compare_df['결과'] = compare_df.apply(lambda row: '일치' if row.nunique() == 1 else '값이다름', axis=1)

# 비교 결과를 "compare.xlsx" 파일로 저장
compare_df.to_excel('compare.xlsx')

# 프로그램 종료 후 메시지 창 표시
showinfo('완료', '매칭이 완료 되었습니다. compare.xlsx 파일을 확인해보세요')


'ok'